# Построение бинарного классификатора токсичных комментариев с использованием логистической регрессии

Описание: решаем задачу бинарной классификации токсичных комментариев на основе датасета Jigsaw с Kaggle.

In [79]:
import pandas as pd
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.utils import resample
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

Для начала посмотрим, что тренировочный датасет из себя представляет:

In [80]:
train_df = pd.read_csv('data/train.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


Токсичные комментарии изначально были поделены на несколько классов, но для бинарной классификации достаточно их все объединить:

In [81]:
train_df['label'] = train_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)
train_df = train_df[['comment_text', 'label']]
train_df.head()

,comment_text,label
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [82]:
train_df[train_df['label'] == 0]['comment_text'].isna().sum()

np.int64(0)

In [83]:
train_df['label'].value_counts()

,count
label,
0,143346
1,16225


Видно, что датасет несбалансирован (почти 90% нетоксичных комментариев), поэтому наша модель сможет всегда предсказывать 0 и получать высокую точность.

Поэтому применим undersampling - уменьшим число нетоксичных комментариев:

In [84]:
train_df_toxic = train_df[train_df['label'] == 1]
train_df_not_toxic = train_df[train_df['label'] == 0]

train_df_undersample = train_df_not_toxic.sample(n=train_df_toxic['label'].sum(), random_state=42)

train_df_not_toxic_under = resample(
    train_df_not_toxic,
    replace=False,
    n_samples=len(train_df_toxic),
    random_state=42
)

balanced_train_df = pd.concat([train_df_toxic, train_df_not_toxic_under])
print(balanced_train_df['label'].value_counts())

label
1    16225
0    16225
Name: count, dtype: int64


Теперь, когда датасет сбалансирован, теперь предобработаем комментарии:

In [85]:
nltk.download('punkt_tab')
nltk.download('stopwords')

snowball = SnowballStemmer(language='english')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [86]:
def tokenize_sentence(sentence: str):

  # Разбиваем на токены
  tokens = word_tokenize(sentence.lower())

  # Убираем всю пунктуацию и стоп-слова
  tokens = [i for i in tokens if i not in string.punctuation]
  tokens = [i for i in tokens if i not in stop_words]

  # Стемминг
  tokens = [snowball.stem(i) for i in tokens]
  return ' '.join(tokens)

In [87]:
balanced_train_df['clean_text'] = balanced_train_df['comment_text'].apply(tokenize_sentence)

Теперь разделим датасет на обучающую и тестовую выборки и сравним распределение:

In [88]:
train_df, test_df = train_test_split(balanced_train_df, test_size = 500, random_state=42)
print(train_df.shape)
print(test_df.shape)

for sample in [train_df, test_df]:
    print(sample[sample['label'] == 1].shape[0] / sample.shape[0])

(31950, 3)
(500, 3)
0.49984350547730827
0.51


Данные равномерно распределены по выборкам, следовательно наша будущая модель должна адекватно оцениваться на тестовых данных

Применим векторизацию TF-IDF:

In [89]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(balanced_train_df['clean_text'])
y = balanced_train_df['label']

Разделим на обучающую и тестовую выборки, обучим модель Логистической регрессии и оценим качество модели на тренировочном датасете:

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print('Accuracy:', accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8949152542372881
              precision    recall  f1-score   support

           0       0.87      0.92      0.90      3229
           1       0.92      0.87      0.89      3261

    accuracy                           0.89      6490
   macro avg       0.90      0.90      0.89      6490
weighted avg       0.90      0.89      0.89      6490



Теперь загрузим тестовый датасет, предобработаем и построим предсказания на основе обученной ранее модели:

In [93]:
test_df = pd.read_csv('data/test.csv')
test_labels = pd.read_csv('data/test_labels.csv')

test_df = test_df.merge(test_labels, on='id')
test_df = test_df[test_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1) != -1]
test_df['label'] = test_df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].max(axis=1)

test_df = test_df[['comment_text', 'label']]

test_df['clean_text'] = test_df['comment_text'].apply(tokenize_sentence)

X_test_real = vectorizer.transform(test_df['clean_text'])
y_test_real = test_df['label']

y_pred_real = model.predict(X_test_real)

print('Accuracy on test.csv:', accuracy_score(y_test_real, y_pred_real))
print(classification_report(y_test_real, y_pred_real))

Accuracy on test.csv: 0.8563412423020413
              precision    recall  f1-score   support

           0       0.99      0.85      0.91     57735
           1       0.40      0.92      0.56      6243

    accuracy                           0.86     63978
   macro avg       0.69      0.89      0.74     63978
weighted avg       0.93      0.86      0.88     63978



# Вывод:

Итоговый accuracy на train.csv - 0.89

Итоговый accuracy на test.csv - 0.86

Высокий recall (0.92) для токсичных комментариев говорит о том, что модель хорошо их находит.

-------------------------------------------------


Важно также заметить, что для **test.csv** низкий **precision** (0.40) для токсичных комментариев означает, что модель часто ошибочно относит нетоксичные комментарии к токсичным

Также средний **f1-score** для нетоксичных комментариев говорит о том, что из-за низкого **precision** модель страдает в балансировке предсказаний токсичности.

В целом, лучше подойдут другие модели, более сложные - такие, как **RandomForest** или нейронные сети (**LSTM**), но и **Логистическая регрессия** дает неплохой результат.